In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import multivariate_normal

In [ ]:
# function to fit
def func(x, a, b, c):
    return a + b*x + c*x**2

# derivatives w.r.t. the fitting parameters
def dafunc(x, a, b, c):
    return 1
def dbfunc(x, a, b, c):
    return x
def dcfunc(x, a, b, c):
    return x**2

In [ ]:
# generate fitting data
ydata = []
npoints = 400

# prescribe parameters and covariance matrix
popt = np.array([1, 3, 4])
pcov = np.array([[3, -1, 0.15],
                 [-1, 1.3, -0.25],
                 [0.15, -0.25, 1]])

# sample a multivariate normal distribution from the covariance matrix
dist = multivariate_normal(mean=popt, cov=pcov, allow_singular = True)
params = dist.rvs(size=npoints)

# compute randomized data
xdata = np.linspace(0, 4, npoints)
for i in range(npoints):
  y = func(xdata[i], *params[i])
  ydata.append(y)
ydata = np.array(ydata)

# plot data and fit
plt.plot(xdata, ydata, 'b.', label='data')
plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
# Simple linear regression does not recover the covariance matrix. TODO: try maximim likelihood optimization
if False:
  popt, pcov = curve_fit(func, xdata, ydata, p0 = popt )

  chi = (ydata - func(xdata, *popt))
  chi2 = (chi ** 2).sum()
  dof = len(xdata) - len(popt)
  factor = (chi2 / dof)

  pcov *= factor

popt, pcov


In [ ]:
# sample parameter sets using the covariance matrix
dist = multivariate_normal(mean=popt, cov=pcov, allow_singular = True)
samples = dist.rvs(size=1000)

# plot function realizations for all sampled parameter sets
for sample in samples:
  plt.plot(xdata, func(xdata, *sample), 'r-', alpha=0.01)
plt.plot(xdata, ydata, 'b.', label='data')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
# compute standard deviation at x using the covariance matrix
def error(x, a, b, c, cov):
    C = np.array(cov)

    #return a + b*x + c*x**2
    def f(x, a,b,c):
        d = np.array([dafunc(x, a, b, c), 
                      dbfunc(x, a, b, c), 
                      dcfunc(x, a, b, c)])
        
        # return np.matmul(np.matmul(d.T, C), d)
        return np.matmul(d, np.matmul(C, d))
    
    return np.sqrt(np.array([f(xi, *popt) for xi in x]))

In [ ]:
# plot the function
plt.plot(xdata, func(xdata, *popt), 'r-')

# two standard deviations
plt.plot(xdata, func(xdata, *popt) - 2 * error(xdata, *popt, pcov), 'r--')
plt.plot(xdata, func(xdata, *popt) + 2 * error(xdata, *popt, pcov), 'r--')

# plot the data
plt.plot(xdata, ydata, 'b.', label='data')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()